In [1]:
import lsst.daf.butler as dafButler

In [2]:
REPO = "/repo/dc2/"
COLL = "2.2i/runs/test-med-1/w_2021_48/DM-32707"

In [3]:
butler = dafButler.Butler(REPO, collections=[COLL,])

In [4]:
dataId = {"tract": 3828,
          "patch": 45,
          "band": "i",
          "skymap": "DC2"
         }

## The naive way - getting info about the coadd but loading it

In [5]:
deepCoadd = butler.get("deepCoadd", dataId=dataId)

In [6]:
info = deepCoadd.getInfo()
coadd_inputs = info.getCoaddInputs()
coadd_inputs.ccds

<class 'lsst.afw.table.ExposureCatalog'>
    id    bbox_min_x bbox_min_y bbox_max_x bbox_max_y ccd visit  goodpix        weight         filter 
             pix        pix        pix        pix                                                     
--------- ---------- ---------- ---------- ---------- --- ------ -------- ------------------ ---------
204708000          0          0       4071       3999   0 204708   195657  4.317103494236756 i_sim_1.4
204708001          0          0       4071       3999   1 204708 13223867  4.317103494236756 i_sim_1.4
204708002          0          0       4071       3999   2 204708   912366  4.317103494236756 i_sim_1.4
204708004          0          0       4071       3999   4 204708  1586036  4.317103494236756 i_sim_1.4
211478000          0          0       4071       3999   0 211478    31251 1.8104210606843396 i_sim_1.4
433962000          0          0       4071       3999   0 433962  2819671 3.8127695024756227 i_sim_1.4
433962001          0          0 

# A better way - get info about the coadd w/o loading the image

In [7]:
deepCoadd_def = butler.getDeferred("deepCoadd", dataId)

In [8]:
coaddInputs = deepCoadd_def.get(component="coaddInputs")
coaddInputs.ccds

<class 'lsst.afw.table.ExposureCatalog'>
    id    bbox_min_x bbox_min_y bbox_max_x bbox_max_y ccd visit  goodpix        weight         filter 
             pix        pix        pix        pix                                                     
--------- ---------- ---------- ---------- ---------- --- ------ -------- ------------------ ---------
204708000          0          0       4071       3999   0 204708   195657  4.317103494236756 i_sim_1.4
204708001          0          0       4071       3999   1 204708 13223867  4.317103494236756 i_sim_1.4
204708002          0          0       4071       3999   2 204708   912366  4.317103494236756 i_sim_1.4
204708004          0          0       4071       3999   4 204708  1586036  4.317103494236756 i_sim_1.4
211478000          0          0       4071       3999   0 211478    31251 1.8104210606843396 i_sim_1.4
433962000          0          0       4071       3999   0 433962  2819671 3.8127695024756227 i_sim_1.4
433962001          0          0 

If you want slightly better visualization of the table, then convert it to an astropy table.

In [9]:
coaddInputs.ccds.asAstropy()

id,bbox_min_x,bbox_min_y,bbox_max_x,bbox_max_y,ccd,visit,goodpix,weight,filter
,pix,pix,pix,pix,,,,,
int64,int32,int32,int32,int32,int32,int64,int32,float64,str32
204708000,0,0,4071,3999,0,204708,195657,4.317103494236756,i_sim_1.4
204708001,0,0,4071,3999,1,204708,13223867,4.317103494236756,i_sim_1.4
204708002,0,0,4071,3999,2,204708,912366,4.317103494236756,i_sim_1.4
204708004,0,0,4071,3999,4,204708,1586036,4.317103494236756,i_sim_1.4
211478000,0,0,4071,3999,0,211478,31251,1.8104210606843396,i_sim_1.4
433962000,0,0,4071,3999,0,433962,2819671,3.8127695024756227,i_sim_1.4
433962001,0,0,4071,3999,1,433962,7323358,3.8127695024756227,i_sim_1.4
433962003,0,0,4071,3999,3,433962,1197337,3.8127695024756227,i_sim_1.4


## How did I know what to provide for the component?

The rule of thumb is to drop `get` from the method name and use lowercase for the first letter.

E.g.,

- getWcs -> wcs
- getPsf -> psf
- getBBox -> bbox (this is an exception!)

This cell below is a butler-magic that lists all the components.
However, not all components may be applicable to `deepCoadd`.

In [10]:
deepCoadd_def.ref.datasetType.makeAllComponentDatasetTypes()

[DatasetType('deepCoadd.image', {band, skymap, tract, patch}, ImageF, parentStorageClass=ExposureF),
 DatasetType('deepCoadd.mask', {band, skymap, tract, patch}, MaskX, parentStorageClass=ExposureF),
 DatasetType('deepCoadd.variance', {band, skymap, tract, patch}, ImageF, parentStorageClass=ExposureF),
 DatasetType('deepCoadd.wcs', {band, skymap, tract, patch}, Wcs, parentStorageClass=ExposureF),
 DatasetType('deepCoadd.psf', {band, skymap, tract, patch}, Psf, parentStorageClass=ExposureF),
 DatasetType('deepCoadd.photoCalib', {band, skymap, tract, patch}, PhotoCalib, parentStorageClass=ExposureF),
 DatasetType('deepCoadd.visitInfo', {band, skymap, tract, patch}, VisitInfo, parentStorageClass=ExposureF),
 DatasetType('deepCoadd.apCorrMap', {band, skymap, tract, patch}, ApCorr, parentStorageClass=ExposureF),
 DatasetType('deepCoadd.coaddInputs', {band, skymap, tract, patch}, CoaddInputs, parentStorageClass=ExposureF),
 DatasetType('deepCoadd.transmissionCurve', {band, skymap, tract, pat